In [282]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [218]:
df = pd.read_json("../scraping/ams_sale.json")

In [276]:
df.shape

(3976, 78)

In [5]:
df.head()

,address,postcode,city,OVE-Vraagprijs,OVE-Vraagprijs per m²,OVE-Aangeboden sinds,OVE-Status,OVE-Aanvaarding,BOU-Soort woonhuis,BOU-Soort bouw,...,IND-Afmetingen,VEI-Prijs,VEI-Veilingperiode,VEI-Soort veiling,VEI-Veilingpartij,BED-Bedrijfsruimte,BED-Kantoorruimte,BED-Winkelruimte,IND-Perceel,BOU-Soort object
0,Buiksloterbreek 41,1034 XC,Amsterdam,€ 475.000 kosten koper,€ 5.220,26 september 2020,Beschikbaar,In overleg,"Eengezinswoning, geschakelde woning",Bestaande bouw,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arènpalmstraat 16,1104 DB,Amsterdam,€ 425.000 kosten koper,€ 3.400,17 september 2020,Beschikbaar,In overleg,"Eengezinswoning, hoekwoning",Bestaande bouw,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brigantijnkade 46,1086 VB,Amsterdam,€ 625.000 kosten koper,€ 4.340,18 september 2020,Beschikbaar,In overleg,"Villa, vrijstaande woning (waterwoning)",Bestaande bouw,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Bosplaat 25,1025 AR,Amsterdam,€ 700.000 kosten koper,€ 5.833,19 september 2020,Beschikbaar,In overleg,"Bungalow, tussenwoning (semi-bungalow)",Bestaande bouw,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Ben van Meerendonkstraat 156,1087 LN,Amsterdam,€ 779.000 kosten koper,€ 4.839,18 september 2020,Beschikbaar,In overleg,"Herenhuis, tussenwoning",Bestaande bouw,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [170]:
df.columns

Index(['address', 'postcode', 'city', 'OVE-Vraagprijs',
       'OVE-Vraagprijs per m²', 'OVE-Aangeboden sinds', 'OVE-Status',
       'OVE-Aanvaarding', 'BOU-Soort woonhuis', 'BOU-Soort bouw',
       'BOU-Bouwjaar', 'BOU-Soort dak', 'OPP-', 'OPP-Perceel', 'OPP-Inhoud',
       'IND-Aantal kamers', 'IND-Aantal badkamers',
       'IND-Badkamervoorzieningen', 'IND-Aantal woonlagen',
       'IND-Voorzieningen', 'ENE-Energielabel', 'ENE-Isolatie',
       'ENE-Verwarming', 'ENE-Warm water', 'ENE-Cv-ketel', 'KAD-', 'BUI-Tuin',
       'BUI-Balkon/dakterras', 'GAR-Soort garage', 'GAR-Capaciteit',
       'GAR-Voorzieningen', 'PAR-Soort parkeergelegenheid', 'BUI-Ligging',
       'BUI-Voortuin', 'BUI-Ligging tuin', 'OVE-Servicekosten',
       'BOU-Specifiek', 'BUI-Zonneterras', 'BER-Schuur/berging',
       'BER-Voorzieningen', 'BUI-Achtertuin', 'BER-Isolatie', 'BOU-Keurmerken',
       'BUI-Patio/atrium', 'OVE-Bijdrage VvE', 'BOU-Soort appartement',
       'BOU-Bouwperiode', 'IND-Gelegen op', 'VVE-In

In [268]:
df.isna().sum()

address                     0
postcode                    0
city                        0
asking_price                0
price_m2                    0
days online                 0
status                      0
acceptance                  4
service_fees_pm          2995
vve_contribution            0
asking_price_original    3829
rent_price               3960
rental_agreement         3960
rent_price_original      3974
sale_type                3953
property_type            3337
new_build                   0
build_year                256
roof_type                1614
specials                 3110
certificates             3637
apartment_type            639
build_era                3720
accessibility            3756
prop_extra_type          3976
parking_type             3976
prop_build_area          3976
opp                         0
property_area            3417
property_volume             0
num_rooms                   0
num_bathrooms             403
bathroom_features         798
floors    

In [267]:
df.dtypes

address                   object
postcode                  object
city                      object
asking_price               int32
price_m2                   int32
days online               object
status                    object
acceptance                object
service_fees_pm           object
vve_contribution         float64
asking_price_original     object
rent_price                object
rental_agreement          object
rent_price_original       object
sale_type                 object
property_type             object
new_build                 object
build_year                object
roof_type                 object
specials                  object
certificates              object
apartment_type            object
build_era                 object
accessibility             object
prop_extra_type           object
parking_type              object
prop_build_area           object
opp                       object
property_area             object
property_volume           object
num_rooms 

In [230]:
def convert_price(x):
    """Return column with integer of price in column."""
    return pd.to_numeric(x.str.extract(r"€ (\S+)", expand=False)
                         .str.replace(".", "")
                         .str.replace(",", ".")
                         .fillna(0))


In [147]:
def convert_elapsed_time(x):
    """Return integer of days since listing online."""
    parts = x.split()
    if len(parts) == 3:
        days = (pd.to_datetime("now") - pd.to_datetime(x)).days
    elif len(parts) == 2:
        time_units = {"weken": 7, "maanden": 40}
        days = int(parts[0].strip("+")) * time_units[parts[1].lower()]
    else:
        days = 1

    return int(days)

In [222]:
# Order of categories
col_trans = ["OVE", "BOU", "OPP",
             "IND", "ENE", "BUI",
             "GAR", "BER", "PAR", 
             "VVE", "KAD", "BED",
             "VEI"]

order = [x
         for i, col in enumerate(col_trans)
         for x in df.columns
         if x.startswith(col_trans[i])] 

df = df[["address", "postcode", "city"] + order]

In [223]:
# Rename columns
cols = {'OVE-Vraagprijs': 'asking_price',
        'OVE-Vraagprijs per m²': 'price_m2',
        'OVE-Aangeboden sinds': 'days_online',
        'OVE-Status': 'status',
        'OVE-Aanvaarding': 'acceptance',
        'BOU-Soort woonhuis': 'property_type',
        'BOU-Soort bouw': 'new_build',
        'BOU-Bouwjaar': 'build_year',
        'BOU-Soort dak': 'roof_type',
        'OPP-': 'opp',
        'OPP-Perceel': 'property_area',
        'OPP-Inhoud': 'property_volume',
        'IND-Aantal kamers': 'num_rooms',
        'IND-Aantal badkamers': 'num_bathrooms',
        'IND-Badkamervoorzieningen': 'bathroom_features',
        'IND-Aantal woonlagen': 'floors',
        'IND-Voorzieningen':  'features',
        'ENE-Energielabel': 'energy_label',
        'ENE-Isolatie': 'isolation',
        'ENE-Verwarming': 'heating',
        'ENE-Warm water': 'hot_water',
        'ENE-Cv-ketel': 'boiler',
        'KAD-': 'kadaster',
        'BUI-Tuin': 'garden',
        'BUI-Balkon/dakterras': 'balcony',
        'GAR-Soort garage': 'garage_type',
        'GAR-Capaciteit': 'garage_size',
        'GAR-Voorzieningen': 'garage_features',
        'PAR-Soort parkeergelegenheid': 'parking',
        'BUI-Ligging': 'environment',
        'BUI-Voortuin': 'garden_front',
        'BUI-Ligging tuin': 'garden_orientation',
        'OVE-Servicekosten': 'service_fees_pm',
        'BOU-Specifiek': 'specials',
        'BUI-Zonneterras': 'terrace',
        'BER-Schuur/berging': 'storage_type',
        'BER-Voorzieningen': 'storage_features',
        'BUI-Achtertuin': 'garden_back',
        'BER-Isolatie': 'storage_isolation',
        'BOU-Keurmerken': 'certificates',
        'BUI-Patio/atrium': 'garden_patio',
        'OVE-Bijdrage VvE': 'vve_contribution',
        'BOU-Soort appartement': 'apartment_type',
        'BOU-Bouwperiode': 'build_era',
        'IND-Gelegen op': 'building_orientation',
        'VVE-Inschrijving KvK': 'vve_kvk',
        'VVE-Jaarlijkse vergadering': 'vve_am',
        'VVE-Periodieke bijdrage': 'vve_per_contr',
        'VVE-Reservefonds aanwezig': 'vve_reserve_fund',
        'VVE-Onderhoudsplan': 'vve_maintenance',
        'VVE-Opstalverzekering': 'vve_insurance',
        'GAR-Isolatie': 'garage_isolation',
        'BOU-Toegankelijkheid': 'accessibility',
        'OVE-Oorspronkelijke vraagprijs': 'asking_price_original',
        'ENE-Voorlopig energielabel': 'energy_label_temp',
        'OVE-Huurprijs': 'rent_price',
        'OVE-Huurovereenkomst': 'rental_agreement',
        'BUI-Plaats': 'garden_plaats',
        'BUI-Zijtuin': 'garden_side',
        'OVE-Oorspronkelijke huurprijs': 'rent_price_original',
        'BOU-Soort overig aanbod': 'prop_extra_type',
        'BED-Praktijkruimte': 'comp_practice',
        'OVE-Koopmengvorm': 'sale_type',
        'BOU-Soort parkeergelegenheid': 'parking_type',
        'IND-Capaciteit': 'parking_capacity',
        'IND-Afmetingen': 'prop_extra_dimensions',
        'VEI-Prijs': 'auction_price',
        'VEI-Veilingperiode': 'auction_period',
        'VEI-Soort veiling': 'auction_type',
        'VEI-Veilingpartij': 'auction_party',
        'BED-Bedrijfsruimte': 'company_space',
        'BED-Kantoorruimte': 'office_space',
        'BED-Winkelruimte': 'store_space',
        'IND-Perceel': 'ground_area',
        'BOU-Soort object': 'prop_build_area'}
df.rename(cols, axis=1, inplace=True)

In [279]:
# Initial drop of columns with little meaning
drop = ['status', 'acceptance', 'asking_price_original', 'rent_price',
        'rental_agreement', 'rent_price_original', 'sale_type', 'certificates',
        'accessibility', 'prop_extra_type', 'parking_type', 'prop_build_area', 
        'opp', 'parking_capacity', 'prop_extra_dimensions', 'ground_area', 
        'garden_plaats', 'garden_side', 'garage_size', 'garage_features',
        'garage_isolation', 'kadaster', 'comp_practice', 'company_space', 
        'office_space', 'store_space', 'auction_price', 'auction_period', 
        'auction_type', 'auction_party']
df.drop(columns=drop, inplace=True)

In [219]:
# Drop rows without asking price
df.dropna(subset=["asking_price"], inplace=True)

In [265]:
# Drop listings that are just garages and such
df = (df.drop(df[df["apartment_type"].isna() 
           & df["property_type"].isna()].index)
      .reset_index(drop=True))

In [228]:
# Convert columns in euro to numeric
euro = ["asking_price", "vve_contribution", "service_fees_pm", "price_m2", "asking_price_original"]
for e in euro:
    df[e] = convert_price(df[e])

In [305]:
# Calculate days since posting
df["days_online"] = df["days_online"].apply(convert_elapsed_time)

In [301]:
# Convert to binary
# Current yes/no columns
binary = ["vve_kvk", "vve_am", "vve_reserve_fund", "vve_maintenance", "vve_insurance"]

# Fill NaN with 0
df[["vve_per_contr"] + binary] = df[["vve_per_contr"] + binary].fillna(0)

# Straight forward columns
for col in binary:
    df[col] = np.where(df[col] == "yes", 1, 0)

# Columns containing numeric values as well
df["vve_per_contr"] = np.where(df["vve_per_contr"].str.contains("yes"), 1, 0)

# Other binary oppositions
df["new_build"] = np.where(df["new_build"] == "Nieuwbouw", 1, 0)

In [ ]:
# Investigate build year and/or era
df[df["build_year"].isna() & df["build_era"].notna()]

In [319]:
def build_era(x):
    """Return mean of build time period."""
    
    if x != x:
        return pd.NA
    start, end = x.split("-")
    
    return int((int(start) + int(end)) / 2)

In [320]:
# Calculate mean of build period
df["build_era"] = (df["build_era"]
                   .apply(build_era)
                   .astype(int, errors="ignore"))

In [334]:
# Use mean of build period if build year is null
df["build_year"] = np.where(df["build_year"].notnull(), df["build_year"], df["build_era"])
df.drop(columns=["build_era"], inplace=True)

In [354]:
a = df["build_year"].str.extract(r"(\d+)", expand=False)
#df["build_year"].unique()

In [355]:
a.isna().sum()

256

In [353]:
df.iloc[3972]["build_year"]

2005

In [335]:
df.sample(20)
#df[df["property_area"].notna()]

,address,postcode,city,asking_price,price_m2,days_online,service_fees_pm,vve_contribution,property_type,new_build,build_year,roof_type,specials,apartment_type,property_area,property_volume,num_rooms,num_bathrooms,bathroom_features,floors,features,building_orientation,energy_label,isolation,heating,hot_water,boiler,energy_label_temp,garden,balcony,environment,garden_front,garden_orientation,terrace,garden_back,garden_patio,garage_type,storage_type,storage_features,storage_isolation,parking,vve_kvk,vve_am,vve_per_contr,vve_reserve_fund,vve_maintenance,vve_insurance
3797,Wittgensteinlaan 261,1062 KG,Amsterdam,425000,4885,42,€ 150 per maand,150.0,NaN,0,1991,NaN,NaN,Bovenwoning (appartement),NaN,259 m³,4 kamers (3 slaapkamers),1 apart toilet,NaN,1 woonlaag,Lift en schuifpui,4e woonlaag,C,Dubbel glas,Cv-ketel,Cv-ketel,"Gas gestookt combiketel, eigendom",NaN,NaN,Balkon aanwezig,"Aan park, aan rustige weg en in woonwijk",NaN,NaN,NaN,NaN,NaN,Parkeerplaats,Box,NaN,NaN,Op afgesloten terrein,0,0,0,0,0,0
2167,Entrepotdok 176,1018 AD,Amsterdam,610000,6932,10,NaN,298.0,NaN,0,Voor 1906,Plat dak bedekt met bitumineuze dakbedekking,Gedeeltelijk gestoffeerd,Bovenwoning (appartement),NaN,255 m³,2 kamers (1 slaapkamer),1 badkamer en 1 apart toilet,Douche,1 woonlaag,Lift,4e woonlaag,B,"Dakisolatie, dubbel glas, muurisolatie, vloeri...",Cv-ketel,Cv-ketel,"Intergas (gas gestookt combiketel, eigendom)",NaN,NaN,Balkon aanwezig,"Aan rustige weg, aan vaarwater, aan water, in ...",NaN,NaN,NaN,NaN,NaN,"Inpandig, parkeerkelder en parkeerplaats",Box,NaN,NaN,NaN,0,0,0,0,0,0
2904,Bouwnummer (Bouwnr. 4),1018 Amsterdam,,560000,7000,240,NaN,0.0,NaN,1,2022,NaN,NaN,Bovenwoning (appartement),NaN,240 m³,3 kamers,NaN,NaN,1 woonlaag,Lift,1e woonlaag,NaN,Volledig geïsoleerd,Warmtepomp,NaN,NaN,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1,0,0,0
3483,Martini van Geffenstraat 134,1068 GL,Amsterdam,350000,3763,10,€ 142 per maand,0.0,NaN,0,2002,Plat dak,NaN,Bovenwoning (appartement),NaN,285 m³,3 kamers (2 slaapkamers),1 badkamer en 1 apart toilet,Ligbad en douche,1 woonlaag,"Lift, mechanische ventilatie en TV kabel",3e woonlaag,B,Volledig geïsoleerd,Cv-ketel,Cv-ketel,"Gas gestookt combiketel uit 2016, eigendom",NaN,NaN,NaN,In woonwijk,NaN,NaN,NaN,NaN,NaN,NaN,Inpandig,Elektra,NaN,"Betaald parkeren, openbaar parkeren en parkeer...",0,0,0,0,0,0
3328,Prins Hendrikkade 107 3,1011 AJ,Amsterdam,897000,7602,240,NaN,0.0,NaN,0,2017,Samengesteld dak bedekt met pannen,Monumentaal pand,Bovenwoning (appartement),NaN,401 m³,4 kamers (3 slaapkamers),2 badkamers,Ligbad en toilet,2 woonlagen,Dakraam en TV kabel,NaN,NaN,"Dakisolatie, dubbel glas, muurisolatie, vloeri...",Cv-ketel,Cv-ketel,"Intergas (gas gestookt combiketel uit 2017, ei...",F,Zonneterras,Dakterras aanwezig,NaN,NaN,Gelegen op het noordwesten,"8 m² (3,43 meter diep en 2,27 meter breed)",NaN,NaN,NaN,NaN,NaN,NaN,Betaald parkeren en parkeervergunningen,0,0,0,0,0,0
1419,Derkinderenstraat 5 A,1062 BE,Amsterdam,500000,4425,21,NaN,0.0,NaN,0,2003,Plat dak bedekt met bitumineuze dakbedekking,NaN,Benedenwoning (appartement),NaN,395 m³,4 kamers (3 slaapkamers),1 badkamer en 1 apart toilet,Ligbad,2 woonlagen,TV kabel,Begane grond,A,Dubbel glas,Cv-ketel,Cv-ketel,"Gas gestookt, eigendom",NaN,Achtertuin,Balkon aanwezig,"Aan rustige weg, in woonwijk en vrij uitzicht",NaN,NaN,NaN,NaN,NaN,Parkeerkelder,Inpandig,NaN,NaN,Betaald parkeren en openbaar parkeren,0,0,0,0,0,0
3730,Quellijnstraat 7 B,1072 XM,Amsterdam,550000,7432,80,€ 97 per maand,97.0,NaN,0,1879,Plat dak bedekt met bitumineuze dakbedekking,Gestoffeerd,Tussenverdieping (appartement),NaN,350 m³,3 kamers (2 slaapkamers),1 badkamer en 1 apart toilet,Douche,1 woonlaag,Mechanische ventilatie,2e woonlaag,C,Dubbel glas,Cv-ketel,Cv-ketel,"Remeha Avanta 28c CR (gas gestookt combiketel,...",NaN,NaN,Balkon aanwezig,Aan rustige weg en in woonwijk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parkeervergunningen,0,0,0,0,0,0
2072,Tolstraat 138 1L,1074 VM,Amsterdam,300000,7895,80,NaN,64.0,NaN,0,Voor 